# 🔍 스테레오 매칭 알고리즘 심화 실습

이 노트북에서는 스테레오 매칭의 핵심 알고리즘들을 직접 구현하고 비교분석해보겠습니다.

## 🎯 학습 목표
- SAD, SSD, Census Transform 알고리즘 직접 구현
- 각 알고리즘의 특징과 장단점 이해
- 처리 시간과 정확도의 트레이드오프 분석
- Winner-Takes-All 방법 구현
- 실제 이미지에서의 성능 비교

## 📦 라이브러리 import 및 설정

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 10)

print("✅ 라이브러리 import 완료!")
print("🔬 스테레오 매칭 알고리즘 구현 준비 완료!")

## 🎨 향상된 테스트 이미지 생성

더 다양한 패턴과 텍스처를 가진 테스트 이미지를 생성합니다.

In [ ]:
def create_enhanced_test_images():
    """
    향상된 테스트 스테레오 이미지 쌍 생성
    다양한 텍스처와 패턴으로 더 현실적인 테스트 환경 제공
    """
    height, width = 300, 400
    
    # 좌측 이미지 생성
    left_img = np.zeros((height, width), dtype=np.uint8)
    
    # 배경 그라디언트
    for i in range(height):
        left_img[i, :] = int(50 + (i / height) * 100)
    
    # 다양한 시차를 가진 객체들
    # 가까운 객체 (큰 시차 - 20픽셀)
    cv2.rectangle(left_img, (80, 80), (150, 150), 255, -1)
    cv2.putText(left_img, 'NEAR', (85, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 0, 2)
    
    # 중간 거리 객체 (중간 시차 - 12픽셀)
    cv2.circle(left_img, (280, 100), 40, 200, -1)
    cv2.circle(left_img, (280, 100), 20, 100, -1)
    
    # 먼 객체 (작은 시차 - 6픽셀)
    cv2.rectangle(left_img, (50, 200), (180, 280), 180, -1)
    cv2.putText(left_img, 'FAR', (85, 245), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 0, 2)
    
    # 텍스처 패턴 추가
    for i in range(220, 280, 10):
        cv2.line(left_img, (200, i), (350, i), 160, 2)
    
    # 체스보드 패턴
    for i in range(8, 12):
        for j in range(32, 38):
            if (i + j) % 2 == 0:
                left_img[i*8:(i+1)*8, j*8:(j+1)*8] = 220
    
    # 우측 이미지 생성 (시차 적용)
    right_img = np.zeros((height, width), dtype=np.uint8)
    
    # 배경 복사
    right_img = left_img.copy()
    
    # 객체들을 시차만큼 이동
    right_img.fill(0)
    
    # 배경 그라디언트 다시 생성
    for i in range(height):
        right_img[i, :] = int(50 + (i / height) * 100)
    
    # 가까운 객체 (20픽셀 왼쪽 이동)
    cv2.rectangle(right_img, (60, 80), (130, 150), 255, -1)
    cv2.putText(right_img, 'NEAR', (65, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 0, 2)
    
    # 중간 거리 객체 (12픽셀 왼쪽 이동)
    cv2.circle(right_img, (268, 100), 40, 200, -1)
    cv2.circle(right_img, (268, 100), 20, 100, -1)
    
    # 먼 객체 (6픽셀 왼쪽 이동)
    cv2.rectangle(right_img, (44, 200), (174, 280), 180, -1)
    cv2.putText(right_img, 'FAR', (79, 245), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 0, 2)
    
    # 텍스처 패턴 (약간 이동)
    for i in range(220, 280, 10):
        cv2.line(right_img, (192, i), (342, i), 160, 2)
    
    # 체스보드 패턴 (이동)
    for i in range(8, 12):
        for j in range(31, 37):  # 1픽셀 왼쪽 이동
            if (i + j) % 2 == 0:
                if j*8 < width-8:
                    right_img[i*8:(i+1)*8, j*8:(j+1)*8] = 220
    
    # 현실적인 노이즈 추가
    noise_left = np.random.normal(0, 10, (height, width))
    noise_right = np.random.normal(0, 10, (height, width))
    
    left_img = np.clip(left_img.astype(np.float32) + noise_left, 0, 255).astype(np.uint8)
    right_img = np.clip(right_img.astype(np.float32) + noise_right, 0, 255).astype(np.uint8)
    
    return left_img, right_img

# 테스트 이미지 생성
left_image, right_image = create_enhanced_test_images()

# 이미지 시각화
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Enhanced Left Image\n(향상된 좌측 이미지)', fontsize=14)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Enhanced Right Image\n(향상된 우측 이미지)', fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

print("✅ 향상된 테스트 이미지 생성 완료!")
print(f"📐 이미지 크기: {left_image.shape}")
print("🎨 특징: 다양한 시차의 객체, 텍스처 패턴, 현실적 노이즈")

## 🧱 스테레오 매칭 기본 클래스 구현

In [ ]:
class StereoMatcher:
    """
    스테레오 매칭 알고리즘들을 구현한 클래스
    """
    
    def __init__(self, max_disparity=32, window_size=5):
        """
        초기화
        
        Parameters:
        - max_disparity: 최대 시차 범위
        - window_size: 윈도우 크기 (홀수)
        """
        if window_size % 2 == 0:
            raise ValueError("윈도우 크기는 홀수여야 합니다.")
        
        self.max_disparity = max_disparity
        self.window_size = window_size
        self.half_window = window_size // 2
        
        print(f"🔧 StereoMatcher 초기화 완료")
        print(f"   - 최대 시차: {max_disparity}")
        print(f"   - 윈도우 크기: {window_size}x{window_size}")
    
    def _get_valid_region(self, img_shape):
        """
        유효한 계산 영역 반환
        """
        h, w = img_shape
        start_y = self.half_window
        end_y = h - self.half_window
        start_x = self.half_window + self.max_disparity
        end_x = w - self.half_window
        
        return start_y, end_y, start_x, end_x

# 매처 객체 생성
matcher = StereoMatcher(max_disparity=32, window_size=5)

print("\n🎯 구현할 알고리즘:")
print("   1. 📊 SAD (Sum of Absolute Differences)")
print("   2. 📈 SSD (Sum of Squared Differences)")
print("   3. 🎭 Census Transform")
print("   4. 🏆 Winner-Takes-All 방법")

## 📊 SAD (Sum of Absolute Differences) 구현

In [ ]:
def compute_sad_disparity(left_img, right_img, max_disparity, window_size, show_progress=True):
    """
    SAD 알고리즘으로 시차 맵 계산
    
    Parameters:
    - left_img, right_img: 좌우 이미지
    - max_disparity: 최대 시차
    - window_size: 윈도우 크기
    - show_progress: 진행률 표시 여부
    """
    h, w = left_img.shape
    half_window = window_size // 2
    disparity = np.zeros((h, w), dtype=np.float32)
    
    # 유효 영역 계산
    start_y = half_window
    end_y = h - half_window
    start_x = half_window + max_disparity
    end_x = w - half_window
    
    total_pixels = (end_y - start_y) * (end_x - start_x)
    processed_pixels = 0
    
    print("🔄 SAD 계산 시작...")
    start_time = time.time()
    
    for y in range(start_y, end_y):
        for x in range(start_x, end_x):
            # 좌측 윈도우 추출
            left_window = left_img[y-half_window:y+half_window+1,
                                 x-half_window:x+half_window+1].astype(np.float32)
            
            min_sad = float('inf')
            best_disparity = 0
            
            # 시차 범위에서 최적 매칭점 찾기
            for d in range(max_disparity):
                if x - d - half_window < 0:
                    break
                
                # 우측 윈도우 추출
                right_window = right_img[y-half_window:y+half_window+1,
                                       x-d-half_window:x-d+half_window+1].astype(np.float32)
                
                # SAD 계산
                sad = np.sum(np.abs(left_window - right_window))
                
                if sad < min_sad:
                    min_sad = sad
                    best_disparity = d
            
            disparity[y, x] = best_disparity
            processed_pixels += 1
            
            # 진행률 표시
            if show_progress and processed_pixels % 1000 == 0:
                progress = (processed_pixels / total_pixels) * 100
                elapsed_time = time.time() - start_time
                print(f"\r진행률: {progress:.1f}% ({processed_pixels}/{total_pixels}) - {elapsed_time:.1f}s", end='')
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ SAD 계산 완료! (소요시간: {elapsed_time:.2f}초)")
    
    return disparity, elapsed_time

# SAD 알고리즘 실행
sad_disparity, sad_time = compute_sad_disparity(left_image, right_image, 32, 5)

# 결과 시각화
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image', fontsize=12)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image', fontsize=12)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(sad_disparity, cmap='plasma')
plt.title(f'SAD Disparity Map\n(처리시간: {sad_time:.2f}초)', fontsize=12)
plt.colorbar(label='Disparity (pixels)')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n📊 SAD 결과 통계:")
valid_sad = sad_disparity[sad_disparity > 0]
if len(valid_sad) > 0:
    print(f"   - 최소 시차: {valid_sad.min():.2f}")
    print(f"   - 최대 시차: {valid_sad.max():.2f}")
    print(f"   - 평균 시차: {valid_sad.mean():.2f}")
    print(f"   - 유효 픽셀: {len(valid_sad):,}개")

## 📈 SSD (Sum of Squared Differences) 구현

In [ ]:
def compute_ssd_disparity(left_img, right_img, max_disparity, window_size, show_progress=True):
    """
    SSD 알고리즘으로 시차 맵 계산
    """
    h, w = left_img.shape
    half_window = window_size // 2
    disparity = np.zeros((h, w), dtype=np.float32)
    
    start_y = half_window
    end_y = h - half_window
    start_x = half_window + max_disparity
    end_x = w - half_window
    
    total_pixels = (end_y - start_y) * (end_x - start_x)
    processed_pixels = 0
    
    print("🔄 SSD 계산 시작...")
    start_time = time.time()
    
    for y in range(start_y, end_y):
        for x in range(start_x, end_x):
            left_window = left_img[y-half_window:y+half_window+1,
                                 x-half_window:x+half_window+1].astype(np.float32)
            
            min_ssd = float('inf')
            best_disparity = 0
            
            for d in range(max_disparity):
                if x - d - half_window < 0:
                    break
                
                right_window = right_img[y-half_window:y+half_window+1,
                                       x-d-half_window:x-d+half_window+1].astype(np.float32)
                
                # SSD 계산: 차이의 제곱의 합
                diff = left_window - right_window
                ssd = np.sum(diff * diff)
                
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_disparity = d
            
            disparity[y, x] = best_disparity
            processed_pixels += 1
            
            if show_progress and processed_pixels % 1000 == 0:
                progress = (processed_pixels / total_pixels) * 100
                elapsed_time = time.time() - start_time
                print(f"\r진행률: {progress:.1f}% ({processed_pixels}/{total_pixels}) - {elapsed_time:.1f}s", end='')
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ SSD 계산 완료! (소요시간: {elapsed_time:.2f}초)")
    
    return disparity, elapsed_time

# SSD 알고리즘 실행
ssd_disparity, ssd_time = compute_ssd_disparity(left_image, right_image, 32, 5)

# SAD vs SSD 비교 시각화
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(sad_disparity, cmap='plasma')
plt.title(f'SAD Result\n처리시간: {sad_time:.2f}초', fontsize=12)
plt.colorbar(label='Disparity')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(ssd_disparity, cmap='plasma')
plt.title(f'SSD Result\n처리시간: {ssd_time:.2f}초', fontsize=12)
plt.colorbar(label='Disparity')
plt.axis('off')

plt.subplot(1, 3, 3)
diff_map = np.abs(sad_disparity - ssd_disparity)
plt.imshow(diff_map, cmap='hot')
plt.title('Difference Map\n(SAD - SSD)', fontsize=12)
plt.colorbar(label='Difference')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n🔍 SAD vs SSD 비교:")
print(f"   ⏱️ 처리 시간 - SAD: {sad_time:.2f}초, SSD: {ssd_time:.2f}초")
print(f"   📊 평균 차이: {np.mean(diff_map[diff_map > 0]):.2f} 픽셀")
print(f"   📈 최대 차이: {np.max(diff_map):.2f} 픽셀")

print(f"\n💡 알고리즘 특성:")
print(f"   📊 SAD: 절댓값 사용, 계산 간단, 노이즈에 민감")
print(f"   📈 SSD: 제곱 사용, 큰 차이에 더 민감, 노이즈 증폭")

## 🎭 Census Transform 구현

Census Transform은 조명 변화와 노이즈에 강한 특징을 가진 알고리즘입니다.

In [ ]:
def census_transform(img, window_size):
    """
    Census Transform 적용
    
    Parameters:
    - img: 입력 이미지
    - window_size: 윈도우 크기
    
    Returns:
    - census: Census transform된 이미지
    """
    h, w = img.shape
    half_window = window_size // 2
    census = np.zeros((h, w), dtype=np.uint32)
    
    print("🎭 Census Transform 적용 중...")
    start_time = time.time()
    
    for y in range(half_window, h - half_window):
        for x in range(half_window, w - half_window):
            center_pixel = img[y, x]
            census_value = 0
            bit_position = 0
            
            # 윈도우 내의 모든 픽셀과 중심 픽셀 비교
            for dy in range(-half_window, half_window + 1):
                for dx in range(-half_window, half_window + 1):
                    if dy == 0 and dx == 0:
                        continue  # 중심 픽셀은 제외
                    
                    neighbor_pixel = img[y + dy, x + dx]
                    if neighbor_pixel < center_pixel:
                        census_value |= (1 << bit_position)
                    bit_position += 1
            
            census[y, x] = census_value
        
        # 진행률 표시
        if y % 20 == 0:
            progress = ((y - half_window) / (h - 2*half_window)) * 100
            print(f"\r진행률: {progress:.1f}%", end='')
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ Census Transform 완료! (소요시간: {elapsed_time:.2f}초)")
    
    return census

def hamming_distance(a, b):
    """
    두 Census 값 간의 해밍 거리 계산
    """
    xor_result = a ^ b
    return bin(xor_result).count('1')

def compute_census_disparity(left_img, right_img, max_disparity, window_size):
    """
    Census Transform을 이용한 시차 맵 계산
    """
    # Census Transform 적용
    print("📝 좌측 이미지 Census Transform...")
    left_census = census_transform(left_img, window_size)
    
    print("📝 우측 이미지 Census Transform...")
    right_census = census_transform(right_img, window_size)
    
    h, w = left_img.shape
    half_window = window_size // 2
    disparity = np.zeros((h, w), dtype=np.float32)
    
    start_y = half_window
    end_y = h - half_window
    start_x = half_window + max_disparity
    end_x = w - half_window
    
    print("🔄 Census 기반 시차 계산 시작...")
    start_time = time.time()
    
    processed_rows = 0
    total_rows = end_y - start_y
    
    for y in range(start_y, end_y):
        for x in range(start_x, end_x):
            left_census_window = left_census[y-half_window:y+half_window+1,
                                           x-half_window:x+half_window+1]
            
            min_cost = float('inf')
            best_disparity = 0
            
            for d in range(max_disparity):
                if x - d - half_window < 0:
                    break
                
                right_census_window = right_census[y-half_window:y+half_window+1,
                                                 x-d-half_window:x-d+half_window+1]
                
                # 해밍 거리 계산
                cost = 0
                for i in range(window_size):
                    for j in range(window_size):
                        cost += hamming_distance(left_census_window[i, j], 
                                               right_census_window[i, j])
                
                if cost < min_cost:
                    min_cost = cost
                    best_disparity = d
            
            disparity[y, x] = best_disparity
        
        processed_rows += 1
        if processed_rows % 10 == 0:
            progress = (processed_rows / total_rows) * 100
            elapsed_time = time.time() - start_time
            print(f"\r진행률: {progress:.1f}% - {elapsed_time:.1f}s", end='')
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ Census 시차 계산 완료! (소요시간: {elapsed_time:.2f}초)")
    
    return disparity, elapsed_time, left_census, right_census

# Census Transform 실행
census_disparity, census_time, left_census, right_census = compute_census_disparity(
    left_image, right_image, 32, 3)  # 작은 윈도우로 계산 시간 단축

print(f"\n🎭 Census Transform 특징:")
print(f"   ✨ 조명 변화에 강함")
print(f"   🛡️ 노이즈에 robust")
print(f"   🔢 비트 패턴 기반 매칭")
print(f"   ⏱️ 계산 시간: {census_time:.2f}초")

## 🎨 Census Transform 결과 시각화

In [ ]:
plt.figure(figsize=(20, 12))

# 원본 이미지들
plt.subplot(2, 4, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Original Left', fontsize=12)
plt.axis('off')

plt.subplot(2, 4, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Original Right', fontsize=12)
plt.axis('off')

# Census Transform 결과
plt.subplot(2, 4, 3)
# Census 값을 시각화를 위해 정규화
left_census_vis = cv2.normalize(left_census.astype(np.float32), None, 0, 255, cv2.NORM_MINMAX)
plt.imshow(left_census_vis, cmap='plasma')
plt.title('Left Census Transform', fontsize=12)
plt.axis('off')

plt.subplot(2, 4, 4)
right_census_vis = cv2.normalize(right_census.astype(np.float32), None, 0, 255, cv2.NORM_MINMAX)
plt.imshow(right_census_vis, cmap='plasma')
plt.title('Right Census Transform', fontsize=12)
plt.axis('off')

# 시차 맵 결과 비교
plt.subplot(2, 4, 5)
plt.imshow(sad_disparity, cmap='viridis')
plt.title(f'SAD Disparity\n{sad_time:.2f}s', fontsize=12)
plt.colorbar()
plt.axis('off')

plt.subplot(2, 4, 6)
plt.imshow(ssd_disparity, cmap='viridis')
plt.title(f'SSD Disparity\n{ssd_time:.2f}s', fontsize=12)
plt.colorbar()
plt.axis('off')

plt.subplot(2, 4, 7)
plt.imshow(census_disparity, cmap='viridis')
plt.title(f'Census Disparity\n{census_time:.2f}s', fontsize=12)
plt.colorbar()
plt.axis('off')

# 알고리즘 비교 요약
plt.subplot(2, 4, 8)
algorithms = ['SAD', 'SSD', 'Census']
times = [sad_time, ssd_time, census_time]
colors = ['blue', 'green', 'red']

bars = plt.bar(algorithms, times, color=colors, alpha=0.7)
plt.ylabel('Processing Time (seconds)')
plt.title('Algorithm Comparison', fontsize=12)
plt.grid(True, alpha=0.3)

# 막대 위에 시간 표시
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{time_val:.2f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("🔍 Census Transform 분석:")
print("   🎨 Census 이미지: 각 픽셀의 비트 패턴을 시각화")
print("   🌈 색상 변화: 서로 다른 텍스처 패턴을 나타냄")
print("   💪 강건성: 조명 변화에도 패턴이 유지됨")

## 📊 알고리즘 성능 정량적 분석

In [ ]:
def analyze_algorithm_performance(disparities, times, names):
    """
    알고리즘 성능 정량적 분석
    """
    print("📊 알고리즘 성능 분석 결과")
    print("=" * 80)
    
    # 헤더 출력
    print(f"{'Algorithm':<12} {'Time(s)':<10} {'Min':<8} {'Max':<8} {'Mean':<8} {'Std':<8} {'Valid%':<8}")
    print("-" * 80)
    
    results = {}
    
    for i, (disparity, time_val, name) in enumerate(zip(disparities, times, names)):
        # 유효한 픽셀만 선택
        valid_pixels = disparity[disparity > 0]
        total_pixels = disparity.size
        valid_ratio = len(valid_pixels) / total_pixels * 100
        
        if len(valid_pixels) > 0:
            min_val = valid_pixels.min()
            max_val = valid_pixels.max()
            mean_val = valid_pixels.mean()
            std_val = valid_pixels.std()
        else:
            min_val = max_val = mean_val = std_val = 0
        
        print(f"{name:<12} {time_val:<10.2f} {min_val:<8.1f} {max_val:<8.1f} {mean_val:<8.1f} {std_val:<8.1f} {valid_ratio:<8.1f}")
        
        results[name] = {
            'time': time_val,
            'min': min_val,
            'max': max_val,
            'mean': mean_val,
            'std': std_val,
            'valid_ratio': valid_ratio,
            'valid_pixels': len(valid_pixels)
        }
    
    return results

# 성능 분석 실행
algorithm_results = analyze_algorithm_performance(
    [sad_disparity, ssd_disparity, census_disparity],
    [sad_time, ssd_time, census_time],
    ['SAD', 'SSD', 'Census']
)

print("\n💡 성능 분석 해석:")
print("   ⏱️ Time: 처리 시간 (낮을수록 좋음)")
print("   📊 Min/Max: 시차 범위 (넓을수록 다양한 깊이 감지)")
print("   📈 Mean: 평균 시차 (데이터 분포 중심)")
print("   📏 Std: 표준편차 (낮을수록 일관된 결과)")
print("   ✅ Valid%: 유효 픽셀 비율 (높을수록 좋음)")

## 🏆 Winner-Takes-All 방법 구현 및 후처리

In [ ]:
def winner_takes_all_with_cost_volume(left_img, right_img, max_disparity, window_size, method='SAD'):
    """
    Winner-Takes-All 방법으로 시차 계산 (비용 볼륨 포함)
    
    Parameters:
    - method: 'SAD', 'SSD', 'NCC' 중 선택
    """
    h, w = left_img.shape
    half_window = window_size // 2
    
    # 비용 볼륨 생성 (높이 x 너비 x 시차)
    cost_volume = np.full((h, w, max_disparity), np.inf, dtype=np.float32)
    disparity = np.zeros((h, w), dtype=np.float32)
    
    print(f"🏆 Winner-Takes-All ({method}) 방법 시작...")
    start_time = time.time()
    
    for y in range(half_window, h - half_window):
        for x in range(half_window + max_disparity, w - half_window):
            left_window = left_img[y-half_window:y+half_window+1,
                                 x-half_window:x+half_window+1].astype(np.float32)
            
            for d in range(max_disparity):
                if x - d - half_window < 0:
                    break
                
                right_window = right_img[y-half_window:y+half_window+1,
                                       x-d-half_window:x-d+half_window+1].astype(np.float32)
                
                # 비용 계산
                if method == 'SAD':
                    cost = np.sum(np.abs(left_window - right_window))
                elif method == 'SSD':
                    diff = left_window - right_window
                    cost = np.sum(diff * diff)
                elif method == 'NCC':
                    # Normalized Cross Correlation
                    left_norm = left_window - np.mean(left_window)
                    right_norm = right_window - np.mean(right_window)
                    
                    numerator = np.sum(left_norm * right_norm)
                    denominator = np.sqrt(np.sum(left_norm**2) * np.sum(right_norm**2))
                    
                    if denominator > 0:
                        ncc = numerator / denominator
                        cost = 1 - ncc  # NCC를 비용으로 변환 (높은 상관관계 = 낮은 비용)
                    else:
                        cost = np.inf
                
                cost_volume[y, x, d] = cost
            
            # Winner-Takes-All: 최소 비용의 시차 선택
            best_disparity = np.argmin(cost_volume[y, x, :])
            disparity[y, x] = best_disparity
        
        if y % 20 == 0:
            progress = ((y - half_window) / (h - 2*half_window)) * 100
            print(f"\r진행률: {progress:.1f}%", end='')
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ Winner-Takes-All ({method}) 완료! (소요시간: {elapsed_time:.2f}초)")
    
    return disparity, cost_volume, elapsed_time

def apply_post_processing(disparity, left_img):
    """
    시차 맵 후처리
    """
    print("🔧 후처리 적용 중...")
    
    # 1. Median filter (노이즈 제거)
    disparity_filtered = cv2.medianBlur(disparity.astype(np.uint8), 5)
    
    # 2. 작은 영역 제거 (morphological opening)
    kernel = np.ones((3,3), np.uint8)
    disparity_opened = cv2.morphologyEx(disparity_filtered, cv2.MORPH_OPEN, kernel)
    
    # 3. 홀 채우기 (morphological closing)
    disparity_closed = cv2.morphologyEx(disparity_opened, cv2.MORPH_CLOSE, kernel)
    
    print("✅ 후처리 완료!")
    
    return disparity_filtered.astype(np.float32), disparity_closed.astype(np.float32)

# Winner-Takes-All with NCC 실행
wta_disparity, cost_volume, wta_time = winner_takes_all_with_cost_volume(
    left_image, right_image, 24, 5, method='NCC')

# 후처리 적용
filtered_disparity, processed_disparity = apply_post_processing(wta_disparity, left_image)

print(f"\n🏆 Winner-Takes-All (NCC) 특징:")
print(f"   🎯 정규화된 상호 상관 사용")
print(f"   💪 조명 변화에 강함")
print(f"   ⏱️ 처리 시간: {wta_time:.2f}초")
print(f"   🔧 후처리로 품질 향상")

## 🎨 최종 결과 종합 비교

In [ ]:
# 모든 결과 종합 시각화
plt.figure(figsize=(20, 15))

# 원본 이미지
plt.subplot(3, 4, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image', fontsize=14)
plt.axis('off')

plt.subplot(3, 4, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image', fontsize=14)
plt.axis('off')

# 알고리즘 결과들
algorithms = [
    (sad_disparity, f'SAD\n{sad_time:.2f}s'),
    (ssd_disparity, f'SSD\n{ssd_time:.2f}s'),
    (census_disparity, f'Census\n{census_time:.2f}s'),
    (wta_disparity, f'NCC (WTA)\n{wta_time:.2f}s'),
    (filtered_disparity, 'Median Filtered'),
    (processed_disparity, 'Morphology Processed')
]

for i, (disp, title) in enumerate(algorithms):
    plt.subplot(3, 4, i + 3)
    plt.imshow(disp, cmap='plasma')
    plt.title(title, fontsize=12)
    plt.colorbar(label='Disparity')
    plt.axis('off')

# 비용 볼륨 시각화 (특정 y 라인)
plt.subplot(3, 4, 9)
y_line = cost_volume.shape[0] // 2
cost_slice = cost_volume[y_line, :, :]
# 무한대 값을 최대값으로 대체
cost_slice_vis = np.where(np.isinf(cost_slice), np.nanmax(cost_slice[~np.isinf(cost_slice)]), cost_slice)
plt.imshow(cost_slice_vis.T, cmap='hot', aspect='auto')
plt.title(f'Cost Volume\n(y={y_line})', fontsize=12)
plt.xlabel('X coordinate')
plt.ylabel('Disparity')
plt.colorbar(label='Cost')

# 처리 시간 비교
plt.subplot(3, 4, 10)
all_algorithms = ['SAD', 'SSD', 'Census', 'NCC']
all_times = [sad_time, ssd_time, census_time, wta_time]
colors = ['blue', 'green', 'red', 'orange']

bars = plt.bar(all_algorithms, all_times, color=colors, alpha=0.7)
plt.ylabel('Processing Time (seconds)')
plt.title('Performance Comparison', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

for bar, time_val in zip(bars, all_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{time_val:.1f}s', ha='center', va='bottom')

# 시차 분포 히스토그램
plt.subplot(3, 4, 11)
for disp, name, color in zip([sad_disparity, ssd_disparity, census_disparity, wta_disparity],
                           ['SAD', 'SSD', 'Census', 'NCC'],
                           ['blue', 'green', 'red', 'orange']):
    valid_disp = disp[disp > 0]
    if len(valid_disp) > 0:
        plt.hist(valid_disp, bins=20, alpha=0.5, label=name, color=color)

plt.xlabel('Disparity (pixels)')
plt.ylabel('Frequency')
plt.title('Disparity Distribution', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# 정확도 비교 (SAD를 기준으로)
plt.subplot(3, 4, 12)
reference = sad_disparity
algorithms_to_compare = [ssd_disparity, census_disparity, wta_disparity]
names_to_compare = ['SSD vs SAD', 'Census vs SAD', 'NCC vs SAD']
colors_compare = ['green', 'red', 'orange']

rmse_values = []
for disp in algorithms_to_compare:
    # 유효한 픽셀만 비교
    valid_mask = (reference > 0) & (disp > 0)
    if np.sum(valid_mask) > 0:
        diff = reference[valid_mask] - disp[valid_mask]
        rmse = np.sqrt(np.mean(diff**2))
        rmse_values.append(rmse)
    else:
        rmse_values.append(0)

bars = plt.bar(names_to_compare, rmse_values, color=colors_compare, alpha=0.7)
plt.ylabel('RMSE (pixels)')
plt.title('Accuracy Comparison\n(vs SAD)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

for bar, rmse_val in zip(bars, rmse_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{rmse_val:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n🎉 종합 분석 완료!")
print("\n📊 결과 요약:")
print(f"   ⚡ 가장 빠른 알고리즘: {all_algorithms[np.argmin(all_times)]} ({min(all_times):.2f}초)")
print(f"   🐌 가장 느린 알고리즘: {all_algorithms[np.argmax(all_times)]} ({max(all_times):.2f}초)")
print(f"   🎯 RMSE 기준 가장 유사: {names_to_compare[np.argmin(rmse_values)]} ({min(rmse_values):.2f})")

print("\n💡 실무 적용 가이드:")
print("   🚀 실시간 처리 필요 → SAD 또는 SSD")
print("   🎨 품질 중시 → Census Transform 또는 NCC")
print("   🌅 조명 변화 환경 → Census Transform")
print("   🔧 노이즈 환경 → 후처리 필수 적용")

## 🧪 실제 이미지 테스트 (선택사항)

In [ ]:
# 실제 스테레오 이미지가 있는 경우 테스트
def test_with_real_images(left_path, right_path):
    """
    실제 스테레오 이미지로 테스트
    """
    try:
        # 이미지 로드
        real_left = cv2.imread(left_path, cv2.IMREAD_GRAYSCALE)
        real_right = cv2.imread(right_path, cv2.IMREAD_GRAYSCALE)
        
        if real_left is None or real_right is None:
            print("❌ 실제 이미지를 찾을 수 없습니다.")
            return
        
        # 크기 조정 (처리 속도를 위해)
        height, width = real_left.shape
        if width > 640:
            scale = 640 / width
            new_width = int(width * scale)
            new_height = int(height * scale)
            real_left = cv2.resize(real_left, (new_width, new_height))
            real_right = cv2.resize(real_right, (new_width, new_height))
        
        print(f"📷 실제 이미지 로드 성공! 크기: {real_left.shape}")
        
        # SAD로 빠른 테스트
        real_disparity, real_time = compute_sad_disparity(
            real_left, real_right, 64, 9, show_progress=False)
        
        # 시각화
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.imshow(real_left, cmap='gray')
        plt.title('Real Left Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(real_right, cmap='gray')
        plt.title('Real Right Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(real_disparity, cmap='plasma')
        plt.title(f'Real Image Disparity\n({real_time:.2f}s)')
        plt.colorbar()
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"✅ 실제 이미지 테스트 완료! 처리시간: {real_time:.2f}초")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 실제 이미지 경로 설정 (있는 경우)
# test_with_real_images('path/to/left.jpg', 'path/to/right.jpg')

print("📝 실제 이미지 테스트:")
print("   위의 test_with_real_images() 함수에 실제 스테레오 이미지 경로를 입력하면")
print("   실제 데이터로 알고리즘을 테스트할 수 있습니다.")
print("   예: test_with_real_images('left.jpg', 'right.jpg')")

## 🎓 실습 요약 및 학습 포인트

In [ ]:
print("🎉 스테레오 매칭 알고리즘 심화 실습 완료!")
print("\n📚 오늘 구현하고 학습한 내용:")
print("   1. 📊 SAD (Sum of Absolute Differences) - 빠르고 간단")
print("   2. 📈 SSD (Sum of Squared Differences) - 큰 차이에 민감")
print("   3. 🎭 Census Transform - 조명 변화에 강함")
print("   4. 🏆 Winner-Takes-All + NCC - 정규화된 상관")
print("   5. 🔧 후처리 기법 - 노이즈 제거 및 품질 향상")

print("\n🔑 핵심 개념:")
print("   💡 매칭 비용: 두 윈도우 간의 유사도/차이도 측정")
print("   🎯 Winner-Takes-All: 최소 비용의 시차 선택")
print("   📦 비용 볼륨: 3차원 비용 공간 (x, y, disparity)")
print("   🔄 후처리: 결과 품질 향상을 위한 필터링")

print("\n⚖️ 알고리즘 특성 비교:")
print("   🚀 속도: SAD > SSD > NCC > Census")
print("   🎨 품질: Census ≈ NCC > SSD ≈ SAD")
print("   💪 강건성: Census > NCC > SSD > SAD")
print("   🔧 구현 난이도: SAD < SSD < NCC < Census")

print("\n🛠️ 실무 선택 가이드:")
print("   📱 모바일/임베디드: SAD + 후처리")
print("   🖥️ 데스크톱 실시간: SSD 또는 NCC")
print("   🎬 오프라인 고품질: Census + 고급 후처리")
print("   🌅 야외 환경: Census (조명 변화 대응)")
print("   🏭 실내 제어 환경: SAD/SSD (빠른 처리)")

print("\n🚀 다음 단계 학습 방향:")
print("   1. 🔧 고급 후처리 기법 (bilateral filter, guided filter)")
print("   2. 🌐 전역 최적화 방법 (graph cuts, belief propagation)")
print("   3. 🤖 딥러닝 기반 스테레오 매칭")
print("   4. 📹 실시간 스테레오 시스템 구축")
print("   5. 🎯 정확도 평가 및 벤치마킹")

print("\n💪 수고하셨습니다! 스테레오 매칭의 핵심을 완전히 이해하셨습니다! 🎉")
print("\n🔗 다음 실습: 깊이 계산 및 3D 재구성으로 이어집니다!")